# Advent of Code 2025

In [41]:
import java.io.File
abstract class Problem(private val filename: String) {
    abstract val testResult: String
    fun run() {
        val testOutput = solve("testData/$filename")
        val actualOutput = solve("data/$filename")
        println("Test results: Expected=$testResult Actual=$testOutput")
        println("Solution: $actualOutput")
    }
    abstract fun solve(path: String): String
}

## Day 1: Secret Entrance
### Part 1
You arrive at the secret entrance to the North Pole base ready to start decorating. Unfortunately, the password seems to have been changed, so you can't get in. A document taped to the wall helpfully explains:

"Due to new security protocols, the password is locked in the safe below. Please see the attached document for the new combination."

The safe has a dial with only an arrow on it; around the dial are the numbers 0 through 99 in order. As you turn the dial, it makes a small click noise as it reaches each number.

The attached document (your puzzle input) contains a sequence of rotations, one per line, which tell you how to open the safe. A rotation starts with an L or R which indicates whether the rotation should be to the left (toward lower numbers) or to the right (toward higher numbers). Then, the rotation has a distance value which indicates how many clicks the dial should be rotated in that direction.

So, if the dial were pointing at `11`, a rotation of `R8` would cause the dial to point at `19`. After that, a rotation of `L19` would cause it to point at `0`.

Because the dial is a circle, turning the dial left from `0` one click makes it point at `99`. Similarly, turning the dial right from `99` one click makes it point at `0`.

So, if the dial were pointing at `5`, a rotation of L10 would cause it to point at `95`. After that, a rotation of `R5` could cause it to point at `0`.

The dial starts by pointing at `50`.

You could follow the instructions, but your recent required official North Pole secret entrance security training seminar taught you that the safe is actually a decoy. The actual password is the number of times the dial is left pointing at `0` after any rotation in the sequence.

For example, suppose the attached document contained the following rotations:
```
L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
```
Following these rotations would cause the dial to move as follows:

The dial starts by pointing at `50`.
The dial is rotated `L68` to point at `82`.
The dial is rotated `L30` to point at `52`.
The dial is rotated `R48` to point at `0`.
The dial is rotated `L5` to point at `95`.
The dial is rotated `R60` to point at `55`.
The dial is rotated `L55` to point at `0`.
The dial is rotated `L1` to point at `99`.
The dial is rotated `L99` to point at `0`.
The dial is rotated `R14` to point at `14`.
The dial is rotated `L82` to point at `32`.
Because the dial points at `0` a total of three times during this process, the password in this example is `3`.

Analyze the rotations in your attached document. What's the actual password to open the door?

In [31]:
class DayOnePartOne(override val testResult: String = "3") : Problem("day1.txt") {
    override fun solve(path: String): String {
        var current = 50
        return File(path).useLines { lines ->
            var zeroCount = 0
            lines.forEach { line ->
                when (line.first()) {
                    'R' -> current += line.drop(1).toInt()
                    'L' -> current -= line.drop(1).toInt()
                }

                if (current % 100 == 0) {
                    zeroCount++
                }
            }
            zeroCount.toString()
        }
    }
}

DayOnePartOne().run()

Test results: Expected=3 Actual=3
Solution: 1139


### Part 2
You're sure that's the right password, but the door won't open. You knock, but nobody answers. You build a snowman while you think.

As you're rolling the snowballs for your snowman, you find another security document that must have fallen into the snow:

"Due to newer security protocols, please use password method 0x434C49434B until further notice."

You remember from the training seminar that "method 0x434C49434B" means you're actually supposed to count the number of times any click causes the dial to point at 0, regardless of whether it happens during a rotation or at the end of one.

Following the same rotations as in the above example, the dial points at zero a few extra times during its rotations:

The dial starts by pointing at `50`.
The dial is rotated `L68` to point at `82`; during this rotation, it points at `0` once.
The dial is rotated `L30` to point at `52`.
The dial is rotated `R48` to point at `0`.
The dial is rotated `L5` to point at `95`.
The dial is rotated `R60` to point at `55`; during this rotation, it points at `0` once.
The dial is rotated `L55` to point at `0`.
The dial is rotated `L1` to point at `99`.
The dial is rotated `L99` to point at `0`.
The dial is rotated `R14` to point at `14`.
The dial is rotated `L82` to point at `32`; during this rotation, it points at `0` once.
In this example, the dial points at `0` three times at the end of a rotation, plus three more times during a rotation. So, in this example, the new password would be `6`.

Be careful: if the dial were pointing at `50`, a single rotation like `R1000` would cause the dial to point at `0` ten times before returning back to `50`!

Using password method 0x434C49434B, what is the password to open the door?

In [40]:
class DayOnePartTwo(override val testResult: String = "6") : Problem("day1.txt") {
    override fun solve(path: String): String {
        var current = 50
        var zeroCount = 0
        File(path).useLines { lines ->
            lines.forEach { line ->
                when (line.first()) {
                    'R' -> {
                        repeat(line.drop(1).toInt()) {
                            current++
                            if (current == 100) {
                                zeroCount++
                                current = 0
                            }
                        }
                    }
                    'L' -> {
                        repeat(line.drop(1).toInt()) {
                            current--
                            when (current) {
                                0 -> zeroCount++
                                -1 -> current = 99
                            }
                        }
                    }
                }
            }
        }
        return zeroCount.toString()
    }
}

DayOnePartTwo().run()

Test results: Expected=6 Actual=6
Solution: 6684


## Day 2: Invalid IDs
### Part 1
You get inside and take the elevator to its only other stop: the gift shop. "Thank you for visiting the North Pole!" gleefully exclaims a nearby sign. You aren't sure who is even allowed to visit the North Pole, but you know you can access the lobby through here, and from there you can access the rest of the North Pole base.

As you make your way through the surprisingly extensive selection, one of the clerks recognizes you and asks for your help.

As it turns out, one of the younger Elves was playing on a gift shop computer and managed to add a whole bunch of invalid product IDs to their gift shop database! Surely, it would be no trouble for you to identify the invalid product IDs for them, right?

They've even checked most of the product ID ranges already; they only have a few product ID ranges (your puzzle input) that you'll need to check. For example:

```
11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124
```
(The ID ranges are wrapped here for legibility; in your input, they appear on a single long line.)

The ranges are separated by commas (,); each range gives its first ID and last ID separated by a dash (-).

Since the young Elf was just doing silly patterns, you can find the invalid IDs by looking for any ID which is made only of some sequence of digits repeated twice. So, `55` (5 twice), `6464` (64 twice), and `123123` (123 twice) would all be invalid IDs.

None of the numbers have leading zeroes; `0101` isn't an ID at all. (`101` is a valid ID that you would ignore.)

Your job is to find all of the invalid IDs that appear in the given ranges. In the above example:

`11-22` has two invalid IDs, `11` and `22`.
`95-115` has one invalid ID, `99`.
`998-1012` has one invalid ID, `1010`.
`1188511880-1188511890` has one invalid ID, `1188511885`.
`222220-222224` has one invalid ID, `222222`.
`1698522-1698528` contains no invalid IDs.
`446443-446449` has one invalid ID, `446446`.
`38593856-38593862` has one invalid ID, `38593859`.
The rest of the ranges contain no invalid IDs.
Adding up all the invalid IDs in this example produces `1227775554`.

What do you get if you add up all of the invalid IDs?

In [27]:
class DayTwoPartOne(override val testResult: String = "1227775554") : Problem("day2.txt") {
    override fun solve(path: String): String {
        var acc = 0L
        File(path).useLines { lines ->
            lines.first().split(",").map { it.split("-") }.forEach { split ->
                for (i in split[0].toLong()..split[1].toLong()) {
                    val str = i.toString()
                    if (str.length % 2 == 0 && str.take(str.length / 2) == str.takeLast(str.length / 2)) {
                        acc += i
                    }
                }
            }
        }
        return acc.toString()
    }
}

DayTwoPartOne().run()

Test results: Expected=1227775554 Actual=1227775554
Solution: 23534117921


### Part 2
The clerk quickly discovers that there are still invalid IDs in the ranges in your list. Maybe the young Elf was doing other silly patterns as well?

Now, an ID is invalid if it is made only of some sequence of digits repeated at least twice. So, `12341234` (`1234` two times), `123123123` (`123` three times), `1212121212` (`12` five times), and `1111111` (`1` seven times) are all invalid IDs.

From the same example as before:

`11-22` still has two invalid IDs, `11` and `22`.
`95-115` now has two invalid IDs, `99` and `111`.
`998-1012` now has two invalid IDs, `999` and `1010`.
`1188511880-1188511890` still has one invalid ID, `1188511885`.
`222220-222224` still has one invalid ID, `222222`.
`1698522-1698528` still contains no invalid IDs.
`446443-446449` still has one invalid ID, `446446`.
`38593856-38593862` still has one invalid ID, `38593859`.
`565653-565659` now has one invalid ID, `565656`.
`824824821-824824827` now has one invalid ID, `824824824`.
`2121212118-2121212124` now has one invalid ID, `121212121`.
Adding up all the invalid IDs in this example produces `4174379265`.

What do you get if you add up all of the invalid IDs using these new rules?

In [30]:
class DayTwoPartTwo(override val testResult: String = "4174379265") : Problem("day2.txt") {
    override fun solve(path: String): String {
        var acc = 0L
        File(path).useLines { lines ->
            lines.first().split(",").map { it.split("-") }.forEach { split ->
                for (i in split[0].toLong()..split[1].toLong()) {
                    if (i < 10) continue
                    val str = i.toString()
                    if (str.all { it == str.first() }) {
                        acc += i
                        continue
                    }

                    for (j in (str.length / 2) downTo 2) {
                        val chunked = str.chunked(j)
                        if (chunked.size > 1 && chunked.all { it == chunked.first() }) {
                            acc += i
                            break
                        }
                    }
                }
            }
        }
        return acc.toString()
    }
}

DayTwoPartTwo().run()

Test results: Expected=4174379265 Actual=4174379265
Solution: 31755323497


## Day 3

### Part 1

In [ ]:
class DayThreePartOne(override val testResult: String = "") : Problem("day3.txt") {
    override fun solve(path: String): String {
        TODO("Not yet implemented")
    }
}